In [1]:
# %pip install XlsxWriter

In [2]:
import xlsxwriter

# Codes need to review or edit

In [3]:
# Dev: replace with output from algo

nClientAuto30id = 2000

nClientInactive = 200
nClientActive = 1800

nClientLPEx = 600
nClientCNPEx = 100
nClientManualEx = 50
nClientCheck = 1050

nClientDataIssue = 150
nClientEgiRun = 900

nClientUnsuc = 200
nClientSuc = 700

nClientMedd = 300
meddLocked = 100
meddPass = 200

nClientCommercial = 400
commLocked = 150
commPass = 250

In [4]:
# To design the structure of the excel 
# Can add more levels based on future needs
# Dev: the key names should be change to a more formal one

report_creation = {
    "first":{
        "Client from auto30id": nClientAuto30id
    },
    "second":{
        "Inactive":nClientInactive, 
        "Active": nClientActive
    },
    "third":{
        "Active LP Exclusions" : nClientLPEx , 
        "Active CNP Exclusions" : nClientCNPEx , 
        "Active Manual Exclusions" : nClientManualEx ,
        "Remaining for Data Quality Check": nClientCheck
    },
    "forth":{
        "Data Issues":nClientDataIssue, 
        "Eligible to Run": nClientEgiRun      
    },
    "fifth":{
        "Unsuccessful": nClientUnsuc,
        "Successful": nClientSuc
    },
    "sixth":{
        "Medd":[{'Medd': nClientMedd}, 
                {"Locked":meddLocked, 
                 "Passthrough":meddPass}], 
        "Commercial":[{"Commercial":nClientCommercial}, 
                      {"Locked":commLocked,
                       "Passthrough":commPass}]
    },

}


#         "Medd":[{'Medd': nClientMedd},
#                 {"Locked":[{'meddLocked':meddLocked},{'brancha':40,'branchb':60}], # this line is for testing whether adding more levels are allowed
#                  "Passthrough":meddPass}], 

<b>How to design the structure<b>
- The structure is a dictionary
- Each key and value pair represents a level of a waterfall. We call it main levels.
- If a level includes more than one value need to drill down. Write sublevels inside the main level. 

<b> How to write sublevels<b>
- The value will be a list instead of a integer.
- In the list, the first item is a dictionary with parent node's name and value; the second item is a dictionary with sublevels' names and values.

In [5]:
# To set file path
filename = "waterfall.xlsx"

In [6]:
workbook = xlsxwriter.Workbook(filename)
worksheet = workbook.add_worksheet()

In [7]:
# To set cells need to be highlight
highlight_title = [
    'Client from auto30id',
    'Active',
    'Remaining for Data Quality Check',
    'Eligible to Run',
    'Successful',
    'Medd',
    'Commercial'
]

In [8]:
# To Set hightlight color
parent_format = workbook.add_format({'bg_color': '#00CC66'})

# Editing is not necessary after this line

In [9]:
def add (levels, row, column): 
    '''
    The function 'add' can help write the N*2 data into the excel file based on the location of the starting cell (row,column).
    Starting cell: the top and left cell for each N*2 data
    
    Parameters
    ---------
    levels: A dictionary. Have the name of the item and the value of it. 
    row: row index for the starting cell (row>=0)
    column: column index for the starting cell (col>=0)
    
    '''
    next_row = row 
    value_column = column + 1    
    
    for i in range(len(levels)):

        # if the value of the dictionary is integer, insert it
        if isinstance(list(levels.values())[i],int):
            worksheet.write(next_row,column,list(levels.keys())[i])
            
            # identify highlight cells
            if list(levels.keys())[i] in highlight_title:
                worksheet.write(next_row,value_column,list(levels.values())[i],parent_format)
            else:
                worksheet.write(next_row,value_column,list(levels.values())[i])    
                
            ## move to the next row to insert a new record
            next_row += 1  
            
        # if the value of dictionary is a list, identify the parent node and values, then recall this function
        else:
            next_row = add(list(levels.values())[i][0],next_row,column)[0]
            next_row = add(list(levels.values())[i][1],next_row,column+1)[0]
                
    # this becomes the next block's start column      
    next_column = value_column 
    
    return next_row,next_column

In [10]:
# To start from (0,0) cell
current_row = 0
current_column = 0

In [11]:
# run the function and insert the contents from report_creation
for i in list(report_creation.keys()):
    current_row,current_column = add(report_creation[i],current_row,current_column)

In [12]:
workbook.close()

In [13]:
# 'xlsxwriter' package doesn't allow opening existing excel file and reset cell format after setting it once.